In [249]:
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
from scipy import sparse
from random import random, choice, randint
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue
from copy import copy
from math import ceil 

def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets


In [250]:
NUM_SETS = 100
NUM_POINTS = 100
DENSITY = .3

#SETS = make_set_covering_problem(NUM_SETS, NUM_POINTS, DENSITY)
SETS = tuple(np.array([1 if random() < DENSITY else 0 for _ in range(NUM_POINTS)]) for _ in range(NUM_SETS)) #Initialize the set

In [251]:
survived = [ 0 for _ in range(NUM_SETS)]
dead = [ 0 for _ in range(NUM_SETS)]

In [252]:
def fitness(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(NUM_POINTS)]),
        )
    )
    return valid, -cost

def avg_sovr(state):
    return sum(
        reduce(
            lambda x,y :  x+y,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(NUM_POINTS)]),
        )
    )/NUM_POINTS

def find_index(state):
    not_found = True
    while(not_found):
        index = randint(0, NUM_POINTS - 1)
        if(state[index]): #SWAP OUT
            if(survived[index] == 0):
                not_found = False
            else:
                not_found = random() >= 1/survived[index]
        else: #SWAP IN
            if(dead[index] == 0):
                not_found = False
            else:
                not_found = random() >= 1/dead[index]
    return index


def tweak(state):
    tmp_state = copy(state)
    for _ in range(1):
        #for _ in range(ceil(avg_sovr(state))):
        index = find_index(tmp_state)
        tmp_state[index] = not tmp_state[index]
    return tmp_state, index

In [253]:
original_state = [choice([False, False, False, False, False, False, False, False, False, True]) for _ in range(NUM_SETS)]

In [254]:
current_state = copy(original_state)

for step in range(1000):
    new_state, index = tweak(current_state)
    if fitness(new_state) >= fitness(current_state):
        current_state = new_state
        print(fitness(current_state))
    else:
        if(current_state[index]):
            survived[index] += 1
        else:
            dead[index] += 1

(99, -13)
(100, -14)
(100, -13)
(100, -12)
(100, -11)
(100, -10)
(100, -9)


In [255]:
def fitness(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(NUM_POINTS)]),
        )
    )
    return valid, -cost

def tweak(state):
    new_state = copy(state)
    index = randint(0, NUM_POINTS - 1)
    new_state[index] = not new_state[index]
    return new_state

In [256]:
current_state_2 = copy(original_state)

for step in range(1000):
    new_state = tweak(current_state_2)
    if fitness(new_state) >= fitness(current_state_2):
        current_state_2 = new_state
        print(fitness(current_state_2))

(99, -13)
(99, -12)
(99, -11)
(100, -12)
(100, -11)
(100, -10)
(100, -9)
(100, -8)


## Se un set viene eliminato dalla soluzione e non c'è nessun miglioramento dev'essere meno probabile sceglierlo la volta successiva per swap out
## Se un set viene portato in soluzione e non apporta nessun miglioramento dev'essere meno probabile sceglierlo la volta successiva per swap in

In [257]:
x = reduce( lambda x,y : x+y, [SETS[i] for i, t in enumerate(current_state) if t],
            np.array([0 for _ in range(NUM_POINTS)]))
print(current_state)
print(sum(x))

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, True, False, True, False, False, False, False, False, True, False, False, True, False, False, False, False, True, False, False, False, True, False, False, False, False, False, False, False, False, True, False, False, True, False, False, False, False, False, False, False]
278
